In [49]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from oauth2client.tools import argparser
import pandas as pd
import re
import urllib.request
import json

# API활용

In [3]:
video_id =  pd.read_csv("D:/경돼님_api/경돼영상url.csv",engine='python')
video_full_id = video_id['url'].apply(lambda x:x.split("=")[1]).tolist()

#개인 api키 입력 필요함.
developer_key = "API키"
youtube_service_name = "youtube"
youtube_api_version = "v3"

youtube_api = build(youtube_service_name,youtube_api_version,developerKey=developer_key)

# 추출가능한 데이터 목록

In [27]:
# snippet에서 뽑아올 수 있는 정보들
title = []
description = []
tags = []
publishedAt = []

# content detail에서 뽑아올 수 있는 정보
duration = []

# statistics에서 뽑아올 수 있는 정보들
view_count = []
likes = []
dislikes = []
comment_count = []

# 데이터 추출

In [31]:
for i in range(len(video_full_id)):
    request = youtube_api.videos().list(
    part = "snippet,contentDetails,statistics",
    id = video_full_id[i]
    )
    response = request.execute()
    
    publishedAt.append(response['items'][0]['snippet']['publishedAt'])
    title.append(response['items'][0]['snippet']['title'])
    try:
        description.append(response['items'][0]['snippet']['description'])
    except KeyError:
        description.append("")
    try:
        tags.append(''.join(response['items'][0]['snippet']['tags']))
    except KeyError:
        tags.append("")
    duration.append(response['items'][0]['contentDetails']['duration'])
    view_count.append(response['items'][0]['statistics']['viewCount'])
    try:
        likes.append(response['items'][0]['statistics']['likeCount'])
    except KeyError:
        likes.append("")
    try:
        dislikes.append(response['items'][0]['statistics']['dislikeCount'])
    except KeyError:
        dislikes.append("")
    try:
        comment_count.append(response['items'][0]['statistics']['commentCount'])
    except KeyError:
        comment_count.append("")

In [140]:
comments_lst = []

In [142]:
for i,j in enumerate(video_full_id):
    video_id_lst2.append(video_full_id[i])
    comments_lst_new = []
    try:
        comments = youtube_api.commentThreads().list(
            part='snippet',
            videoId = video_full_id[i],
            maxResults = 100
        ).execute()
    except HttpError:
        comments_lst.append([])
    while comments:
        for item in comments['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments_lst_new.append(comment['textOriginal'])
        if 'nextPageToken' in comments:
            comments = youtube_api.commentThreads().list(part='snippet',videoId = video_full_id[i],
                       pageToken = comments['nextPageToken'],maxResults = 100).execute()                              
        else:
            comments_lst.append(comments_lst_new)
            break